In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

print(f'GPU available: {torch.cuda.is_available()}')

Sat May  1 14:19:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install emoji

     |████████████████████████████████| 133kB 8.6MB/s 


In [20]:
import StockSentimentDataset
from torch.utils.data import DataLoader
from sklearn.utils.class_weight import compute_class_weight

train_ds, val_ds, test_ds, embeddings, featurizer = StockSentimentDataset.load_all_data()

class_weights = compute_class_weight(class_weight='balanced', classes=[0,1,2], y=train_ds.labels)
class_weights_tensor = torch.FloatTensor(class_weights)

In [15]:
blah4 = torch.ones((3,2))
torch.cat(2*[torch.unsqueeze(blah4,dim=0)], dim=0).size()

torch.Size([2, 3, 2])

In [16]:
blah1 = torch.ones((2, 3, 2))
blah2 = torch.BoolTensor([[False, True, False], [True, False, False]])
blah3 = torch.sum(blah1*torch.unsqueeze(blah2, dim=2), dim=0)
blah3_expanded = torch.cat(blah1.size()[0]*[torch.unsqueeze(blah3,dim=0)], dim=0)
torch.cat([blah1, blah3_expanded], dim=2).size()

torch.Size([2, 3, 4])

In [19]:
blah1 = torch.FloatTensor([[3, 5], [1, 2]])
blah2 = torch.ones((3,2,2))
(blah2 * blah1.view(1,2,2)).size()

torch.Size([3, 2, 2])

In [46]:
import torch.nn.functional as f
class BRCAN(nn.Module):
    def __init__(self, hidden_size, attention_hidden_dim, embedding_dim, 
                 n_filters, vocab_size, pad_token, end_token):
        super(BRCAN, self).__init__()

        self.n_filters = n_filters
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.bilstm = nn.LSTM(embedding_dim, hidden_size, bidirectional=True, batch_first=True)
        self.cnn_2gram = nn.Conv1d(hidden_size*2, out_channels=self.n_filters, kernel_size=2, stride=1)
        self.cnn_3gram = nn.Conv1d(hidden_size*2, out_channels=self.n_filters, kernel_size=3, stride=1)

        self.w_a = nn.Linear(hidden_size*4, attention_hidden_dim)
        self.v_a = nn.Linear(attention_hidden_dim, 2*n_filters)

        self.output_layer = nn.Linear(hidden_size*2, 3)

        self.pad_token = pad_token
        self.end_token = pad_token

        self.lstm_drop = nn.Dropout(0.3)
        self.cnn_drop = nn.Dropout(0.3)
        self.input_drop = nn.Dropout(0.2)


    def forward(self, X, train=False):
        only_end_token_mask = X == self.end_token
        pad_token_mask = X == self.pad_token

        x = self.embeddings(X)
        x = self.input_drop(x)
        # print(x.size())
        lstm_out, lstm_hidden = self.bilstm(x)
        # lstm_out = self.lstm_drop(lstm_out)
        # print(lstm_out.size())

        lstm_out_cnn_in = torch.transpose(torch.transpose(lstm_out, 0, 1), 1, 2)
        lstm_out_2gram_out = f.relu(self.cnn_2gram(lstm_out_cnn_in))
        lstm_out_2gram_out, _ = torch.max(lstm_out_2gram_out, dim=2)
        # print(lstm_out_2gram_out.size())

        lstm_out_3gram_out = f.relu(self.cnn_3gram(lstm_out_cnn_in))
        lstm_out_3gram_out, _ = torch.max(lstm_out_3gram_out, dim=2)
        
        conv_out = torch.cat([lstm_out_2gram_out, lstm_out_3gram_out], dim=1)
        # conv_out = self.cnn_drop(conv_out)
        # print(conv_out.size())

        # mask dim: S, B
        # lstm_out dim: S, B, x*2
        end_token_lstm_out = torch.sum(lstm_out * torch.unsqueeze(only_end_token_mask, dim=2), dim=0)
        end_token_expanded = torch.cat(lstm_out.size()[0]*[torch.unsqueeze(end_token_lstm_out,dim=0)], dim=0)
        w_a_in = torch.cat([lstm_out, end_token_expanded], dim=2)
        w_a_out = f.tanh(self.w_a(w_a_in))
        # print(w_a_out.size())

        v_a_out = self.v_a(w_a_out)
        # print(v_a_out.size())

        # conv_out dim: B, n_filter
        # v_a_out dim: S, B, n_filter
        v_a_out_transposed = torch.transpose(v_a_out, 0, 1) # B, S, n_filter
        attention_scores = torch.bmm(v_a_out_transposed, torch.unsqueeze(conv_out, dim=2)) # B, S, 1
        attention_scores = torch.squeeze(attention_scores) # B, S
        pad_token_mask_transposed = torch.transpose(pad_token_mask, 0, 1) # B, S
        attention_scores[pad_token_mask_transposed] = -float('inf')
        attention_weights = f.softmax(attention_scores, dim=1)

        weighted_lstm_out = lstm_out * torch.unsqueeze(torch.transpose(attention_weights, 0, 1), dim=2)

        lstm_out_combined = torch.sum(weighted_lstm_out, dim=0)

        return self.output_layer(lstm_out_combined)

# train_dl = DataLoader(dataset=train_ds, batch_size=5,
#     shuffle=True, collate_fn=StockSentimentDataset.collate_fn)
# model = BRCAN(100, 100, 100, 
#                  30, featurizer.vocab_size, featurizer.PADDING_ID, featurizer.token_id_lookup[featurizer.END_TOKEN])
# model = model.cuda()
# for x, y in train_dl:
#   x = x.cuda()
#   y = y.cuda()
#   predicted_scores = model(x)
#   print(predicted_scores.size())
#   break


In [31]:
from sklearn.metrics import f1_score
import random

def score_model(model, test_ds):
  model.eval()
  with torch.no_grad():
    test_dl = DataLoader(dataset=test_ds, batch_size=len(test_ds.samples),
      shuffle=False, collate_fn=StockSentimentDataset.collate_fn)
    predictions = []
    for x, y in test_dl:
      x = x.cuda()
      y = y.cuda()
      pred_scores = model(x)
      predictions.extend(torch.argmax(pred_scores, dim=1).tolist())
  return f1_score(test_ds.labels, predictions, labels=[0,1,2], average='micro')


In [47]:
#Training

from random import sample
import tqdm
import os
import subprocess
import random
from pretrained_embeddings import add_embeddings_to_embedding_matrix


def train(model, best_val_model, train_ds, val_ds, class_weights):
  ### Note: not using class_weights in the loss function since that gives worse performance
  class_weights = class_weights.cuda()
  optimizer = optim.Adam(model.parameters(), lr=0.001)
  loss_fn = torch.nn.CrossEntropyLoss()

  train_dl = DataLoader(dataset=train_ds, batch_size=len(train_ds.samples),
    shuffle=True, collate_fn=StockSentimentDataset.collate_fn)

  val_dl = DataLoader(dataset=val_ds, batch_size=len(val_ds.samples),
    shuffle=False, collate_fn=StockSentimentDataset.collate_fn)

  epoch_stats = []
  best_f1 = 0.0
  n_epochs_without_improvement = 0
  best_val_model.load_state_dict(model.state_dict())
  for epoch in range(250):
    totalLoss = 0.0
    n_samples = 0
    model.train()
    for x, y in tqdm.notebook.tqdm(train_dl, leave=False):
      x = x.cuda()
      y = y.cuda()
      model.zero_grad()
      predicted_scores = model(x)
      loss = loss_fn(predicted_scores, y)
      loss.backward()
      totalLoss += loss.item()
      n_samples += x.size()[1]
      optimizer.step()


    average_train_loss = totalLoss / float(n_samples)

    model.eval()
    total_val_loss = 0.0
    n_val_samples = 0
    with torch.no_grad():
      for x, y in val_dl:
        x = x.cuda()
        y = y.cuda()
        predicted_scores = model(x)
        loss = loss_fn(predicted_scores, y)
        total_val_loss += loss.item()
        n_val_samples += x.size()[1]
    
    average_val_loss = total_val_loss / float(n_val_samples)
    epoch_stats.append((epoch, average_train_loss, average_val_loss))
    val_f1 = score_model(model, val_ds)
    if epoch % 1 == 0:
      print("Epoch {0} finished. Train loss: {1}, Val loss: {2}".format(epoch, average_train_loss, average_val_loss))

    if val_f1 > best_f1:
      best_f1 = val_f1
      n_epochs_without_improvement = 0
      best_val_model.load_state_dict(model.state_dict())
    # else:
    #   n_epochs_without_improvement += 1
    #   if n_epochs_without_improvement == 10:
    #     break

  return best_f1, epoch_stats

model = BRCAN(100, 100, 100, 30, featurizer.vocab_size, featurizer.PADDING_ID, featurizer.token_id_lookup[featurizer.END_TOKEN])
best_val_model = BRCAN(100, 100, 100, 30, featurizer.vocab_size, featurizer.PADDING_ID, featurizer.token_id_lookup[featurizer.END_TOKEN])
add_embeddings_to_embedding_matrix(embeddings, model.embeddings, featurizer)
model = model.cuda()
best_val_model = best_val_model.cuda()
train(model, best_val_model, train_ds, val_ds, class_weights_tensor)

Epoch 0 finished. Train loss: 0.0012059610406688862, Val loss: 0.005543525121649917


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1698: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Epoch 1 finished. Train loss: 0.0011935833394855293, Val loss: 0.005519494718434859


Epoch 2 finished. Train loss: 0.0011858136215881414, Val loss: 0.005514530503019995


Epoch 3 finished. Train loss: 0.0011819528930126898, Val loss: 0.005519475863904369


...

Epoch 246 finished. Train loss: 6.227848930458555e-05, Val loss: 0.006791843443500752


Epoch 247 finished. Train loss: 7.632959990611564e-05, Val loss: 0.006857209059656883


Epoch 248 finished. Train loss: 7.661913159919127e-05, Val loss: 0.006945602747858787


Epoch 249 finished. Train loss: 7.522433209340564e-05, Val loss: 0.00692008587778831


(0.5867346938775511,
 [(0, 0.0012059610406688862, 0.005543525121649917),
  (1, 0.0011935833394855293, 0.005519494718434859),
  (2, 0.0011858136215881414, 0.005514530503019995),
  (3, 0.0011819528930126898, 0.005519475863904369),
  (4, 0.0011780078392742228, 0.005523848290346107),
  (5, 0.0011781606212569805, 0.005521170338805841),
  (6, 0.0011766073596228336, 0.005509927564737748),
  (7, 0.001172597521077944, 0.0055010860063591785),
  (8, 0.0011681487457980417, 0.005491900200746498),
  (9, 0.0011600297932053032, 0.005491288340821558),
  (10, 0.0011626141037222313, 0.005486306487297525),
  (11, 0.0011610643829580711, 0.005476465030592315),
  (12, 0.001156327748062587, 0.005483585352800331),
  (13, 0.001155922515164114, 0.0054798174877556),
  (14, 0.0011511909948586107, 0.0054761414625206775),
  (15, 0.0011515694745171843, 0.0054792275234144565),
  (16, 0.0011459758179415964, 0.0054707630556456895),
  (17, 0.0011423912772251042, 0.005471072634872125),
  (18, 0.0011410711884367454, 0.0054

In [48]:
print(score_model(model, train_ds))
print(score_model(model, test_ds))
print(score_model(model, val_ds))

print(score_model(best_val_model, train_ds))
print(score_model(best_val_model, test_ds))
print(score_model(best_val_model, val_ds))


0.9988998899889989
0.5743589743589743
0.5306122448979592
0.9757975797579758
0.6102564102564103
0.5867346938775511


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1698: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
